In [1]:
#import numpy as np
#import pandas as pd
#from scipy.optimize import linprog
#import matplotlib.pyplot as plt

In [3]:
#dataset = pd.read_csv("Lnt Kalyani Input.csv", sep=" ")

In [6]:
import csv
from ortools.linear_solver import pywraplp

# Define the input data
data = [{'Diameter': 12, 'Length': 6, 'Number': 1},
        {'Diameter': 12, 'Length': 5, 'Number': 2},
        {'Diameter': 12, 'Length': 6, 'Number': 2},
        {'Diameter': 12, 'Length': 3, 'Number': 4},
        {'Diameter': 12, 'Length': 3, 'Number': 6},
       ]

with open('Test_Data.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        data.append({'Diameter': int(row['Diameter']), 'Length': float(row['Length']), 'Number': int(row['Number'])})

# Define the required lengths to fulfill
required_lengths = [5, 4, 3]

# Define the rebar length
rebar_length = 12

# Create a solver instance
solver = pywraplp.Solver.CreateSolver('SCIP')

# Define variables
cut_vars = {}
for i, d in enumerate(data):
    cut_vars[i] = {}
    for length in required_lengths:
        cut_vars[i][length] = solver.IntVar(0, d['Number'], f'CutVar_{i}_{length}')

# Define the objective function: maximize total length of cut rebar
solver.Maximize(solver.Sum([cut_vars[i][length] * data[i]['Length'] for i in cut_vars for length in required_lengths]))

# Define constraints: each cut should be less than or equal to rebar length
for i, d in enumerate(data):
    for length in required_lengths:
        solver.Add(cut_vars[i][length] * d['Length'] <= rebar_length)

# Define the column generation function
def generate_column():
    pattern = []
    for d in data:
        for length in required_lengths:
            if d['Length'] < rebar_length and d['Number'] > 0 and length <= rebar_length:
                num_pieces_to_cut = min(d['Number'], int(rebar_length / length))
                pattern.append((length * num_pieces_to_cut, length, num_pieces_to_cut))
                d['Number'] -= num_pieces_to_cut
    return pattern

# Define the column generation loop
while True:
    solver.Solve()

    # Check if the solution is integer
    if all(cut_vars[i][length].solution_value() % 1 == 0 for i in cut_vars for length in required_lengths):
        break

    # Generate a new cutting pattern
    new_pattern = generate_column()

    # Add the new cutting pattern to the model
    if new_pattern:
        new_vars = {}
        for j, (total_length, pattern_length, num_pieces) in enumerate(new_pattern):
            for length in required_lengths:
                new_var = solver.IntVar(0, num_pieces, f'NewVar_{j}_{length}')
                new_vars[(j, length)] = new_var
                solver.Add(new_var == num_pieces)
                solver.Add(new_var * pattern_length <= rebar_length)
                solver.Add(new_var * pattern_length >= rebar_length * (1 - (1 - pattern_length) / rebar_length))
                solver.Add(new_var >= 0)
                solver.Add(cut_vars[data.index({'Diameter': length, 'Length': pattern_length, 'Number': 1})][length] >= new_var)

# Calculate total yield
total_yield = sum(cut_vars[i][length].solution_value() * data[i]['Length'] for i in cut_vars for length in required_lengths)

# Print the results
print('Total Yield:', total_yield)
print('Percentage Yield:', total_yield / (rebar_length * sum(d['Number'] for d in data)) * 100)

# Print cutting patterns
print('Cutting Patterns:')
for i, d in enumerate(data):
    for length in required_lengths:
        print(f"Diameter {d['Diameter']} - Length {length} - Number of Cuts: {cut_vars[i][length].solution_value()}")